In [125]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag
from pyspark.sql.functions import col, when, lit

In [126]:
spark = SparkSession.builder.appName('Popularity Percentage').getOrCreate()

In [127]:
df_input = spark.read.options(header= 'True', InferSchema='True').csv('facebook_friends.csv')
df_input.show()

+-----+-----+
|user1|user2|
+-----+-----+
|    2|    1|
|    1|    3|
|    4|    1|
|    1|    5|
|    1|    6|
|    2|    6|
|    7|    2|
|    8|    3|
|    3|    9|
+-----+-----+



In [138]:
df_input.createOrReplaceTempView("dept")
df_mutliple_user1_records = spark.sql("select user1 from dept where user1 in(select user2 from dept)");
df_mutliple_user2_records = spark.sql("select user2 from dept where user2 in(select user1 from dept)");
df_single_user1_records = spark.sql("select user1 from dept where user1 not in(select user2 from dept)");
df_single_user2_records = spark.sql("select user2 from dept where user2 not in(select user1 from dept)");

In [139]:
df_merged = df_mutliple_user1_records.union(df_mutliple_user2_records).union(df_single_user1_records).union(df_single_user2_records)

In [140]:
df_merged = df_merged.groupBy('user1').count().orderBy('user1').select(col("user1"), col("count").alias("popularity_count"))

In [143]:
## calculating percentage
df_output = df_merged.withColumn("popularity_percent", (df_merged['popularity_count']/df_merged.count())*100)

In [144]:
df_output.selectExpr('user1','popularity_percent').show()

+-----+------------------+
|user1|popularity_percent|
+-----+------------------+
|    1| 55.55555555555556|
|    2| 33.33333333333333|
|    3| 33.33333333333333|
|    4| 11.11111111111111|
|    5| 11.11111111111111|
|    6| 22.22222222222222|
|    7| 11.11111111111111|
|    8| 11.11111111111111|
|    9| 11.11111111111111|
+-----+------------------+

